In [ ]:
# make sure we are working in module directory
repo_root = !git rev-parse --show-toplevel
module_path = repo_root[0] + "/backend/heatflask"
%cd $module_path

import sys
__package__ = "heatflask"
if ".." not in sys.path:
    sys.path.insert(0, "..")

# Make cells wider
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


In [ ]:
%load Strava.py

In [ ]:
# Test Create Client
import logging
logging.basicConfig(level="DEBUG")

C = AsyncClient("admin")
C, C.expires_in

In [ ]:
print("Paste this URL into your browser and retrieve the code:\n", get_auth_url())

In [ ]:
CODE = "325f4ab4e610ce40eaf23ccf8ec15c3141819d62"

In [ ]:
# Test refresh access_token

# exchange code for token
new_info = await C.update_access_token(code=CODE)
C

In [ ]:
# # Force refresh token
new_info = await C.update_access_token(
    stale_ttl=C.expires_in + 1
)
new_info

In [ ]:
async with C:
    U = await C.get_athlete()
U

In [ ]:
log.setLevel("DEBUG")

gen = C.get_index()
first10 =  [await gen.__anext__() for i in range(10)]
A = first10[0]
A

In [ ]:
await AsyncClient.abort(gen)

In [ ]:
aid, streams = await C.get_streams(A["id"])
aid, streams

In [ ]:
ids = [a["id"] for a in first10]
gen = C.get_many_streams(ids)
first5 = [await gen.__anext__() for i in range(5)]
# data = {aid: s async for ais, s in gen}

In [ ]:
await AsyncClient.abort(gen)

In [ ]:
first5

In [ ]:
g = []
g[10] = None 

In [ ]:
a = asyncio.create_task(asyncio.sleep(360), name=4)

In [ ]:
a.done()